In [3]:
pip install azure-eventhub>=5.11.0, Kqlmagic --no-cache-dir --upgrade --quiet

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 4, Finished, Available)

Note: you may need to restart the kernel to use updated packages.


In [4]:
%%capture
reload_ext Kqlmagic

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 5, Finished, Available)

In [9]:
%kql azureDataExplorer://code;cluster='help';database='Samples'

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 10, Finished, Available)

<IPython.core.display.Javascript object>

## Start developing my alert

In [12]:
q="StormEvents | count"

%kql result << -query q
result[0]

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 13, Finished, Available)

<IPython.core.display.Javascript object>

{
    "Count": 59066
}

In [13]:
import json, time
from azure.eventhub import EventHubProducerClient, EventData

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 14, Finished, Available)

In [26]:
#i=json.dumps(result[0])
i=json.dumps({"Count": 0,})

producer = EventHubProducerClient.from_connection_string(conn_str="")

b = producer.create_batch() 
n = 1
for n in range(60):
    b.add(EventData(str(i)))
    producer.send_batch(b) # send it!
    time.sleep(1)
    n=n+1

producer.close()

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 27, Finished, Available)

In [23]:
q="StormEvents | summarize count(), avg(DamageProperty) by State | top 5 by count_ desc"

%kql result << -query q
result

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 24, Finished, Available)

<IPython.core.display.Javascript object>

State,count_,avg_DamageProperty
TEXAS,4701,114170.11274196979
KANSAS,3166,217996.99936828806
IOWA,2337,53604.49293966624
ILLINOIS,2022,16193.372898120673
MISSOURI,2016,203371.7757936508


[{'State': 'TEXAS', 'count_': 4701, 'avg_DamageProperty': 114170.11274196979},
 {'State': 'KANSAS', 'count_': 3166, 'avg_DamageProperty': 217996.99936828806},
 {'State': 'IOWA', 'count_': 2337, 'avg_DamageProperty': 53604.49293966624},
 {'State': 'ILLINOIS', 'count_': 2022, 'avg_DamageProperty': 16193.372898120673},
 {'State': 'MISSOURI', 'count_': 2016, 'avg_DamageProperty': 203371.7757936508}]

### Convert Dict to JSON for multiple rows

In [24]:
import json  

# Data to be written  
dictionary = result.to_dict()

# Serializing json   
json_object = json.dumps(dictionary)  
print(json_object)

df=json.loads(json_object)

StatementMeta(, fd0a7c71-f6d0-43c7-9814-a7cea72bb36a, 25, Finished, Available)

{"State": ["TEXAS", "KANSAS", "IOWA", "ILLINOIS", "MISSOURI"], "count_": [4701, 3166, 2337, 2022, 2016], "avg_DamageProperty": [114170.11274196979, 217996.99936828806, 53604.49293966624, 16193.372898120673, 203371.7757936508]}
